## Cotiviti Internship POC

Kolton Hauck

Utilizing OpenAI's Davinci-003 LLM to identify ICD-10 and RxCUI codes in Free Text.

### Imports

In [1]:
import langchain #interface to LLMs and other modules
import os #get env tokens

from langchain import PromptTemplate #import PromptTemplate for ease of prompt construction
from langchain.llms import OpenAI #access to openai api through langchain

from dotenv import load_dotenv #get env tokens

load_dotenv() #load env tokens
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") #set openai api key to env

### Initialize Model and Sample Texts

In [2]:
llm = OpenAI(model_name="text-davinci-003") #initialize llm -> easy to specify

#sample texts for free-text notes
#samples retrieved from clinical dataset: https://huggingface.co/datasets/Elfsong/ClinicalDataset/viewer/Elfsong--ClinicalDataset
texts = ["He is a 24-year-old male who said that he had gotten into some poison ivy this weekend while he was fishing. He has had several cases of this in the past and he says that is usually takes quite awhile for him to get over it; he said that the last time he was here he got a steroid injection by Dr. Blackman; it looked like it was Depo-Medrol 80 mg. He said that it worked fairly well, although it seemed to still take awhile to get rid of it. He has been using over-the-counter Benadryl as well as cortisone cream on the areas of the rash and having a little bit of improvement, but this last weekend he must have gotten into some more poison ivy because he has got another outbreak along his chest, legs, arms and back.",
"This is an 11-year-old female who comes in for two different things. 1. She was seen by the allergist. No allergies present, so she stopped her Allegra, but she is still real congested and does a lot of snorting. They do not notice a lot of snoring at night though, but she seems to be always like that. 2. On her right great toe, she has got some redness and erythema. Her skin is kind of peeling a little bit, but it has been like that for about a week and a half now.",
"The patient is a 25-year-old right-handed Caucasian female who presented to the emergency department with sudden onset of headache occurring at approximately 11 a.m. on the morning of the July 31, 2008. She described the headache as worse in her life and it was also accompanied by blurry vision and scotoma. The patient also perceived some swelling in her face. Once in the Emergency Department, the patient underwent a very thorough evaluation and examination. She was given the migraine cocktail. Also was given morphine a total of 8 mg while in the Emergency Department. For full details on the history of present illness, please see the previous history and physical.",
"The patient is an 89-year-old lady. She actually turns 90 later this month, seen today for a short-term followup. Actually, the main reasons we are seeing her back so soon which are elevated blood pressure and her right arm symptoms are basically resolved. Blood pressure is better even though she is not currently on the higher dose Mavik likely recommended. She apparently did not feel well with the higher dose, so she just went back to her previous dose of 1 mg daily. She thinks, she also has an element of office hypertension. Also, since she is on Mavik plus verapamil, she could switch over to the combined drug Tarka. However, when we gave her samples of that she thought they were too big for her to swallow. Basically, she is just back on her previous blood pressure regimen. However, her blood pressure seems to be better today. Her daughter says that they do check it periodically and it is similar to today's reading. Her right arm symptoms are basically resolved and she attributed that to her muscle problem back in the right shoulder blade. We did do a C-spine and right shoulder x-ray and those just mainly showed some degenerative changes and possibly some rotator cuff injury with the humeral head quite high up in the glenoid in the right shoulder, but this does not seem to cause her any problems. She has some vague 'stomach problems', although apparently it is improved when she stopped Aleve and she does not have any more aches or pains off Aleve. She takes Tylenol p.r.n., which seems to be enough for her. She does not think she has any acid reflux symptoms or heartburn. She does take Tums t.i.d. and also Mylanta at night. She has had dentures for many, many years and just recently I guess in the last few months, although she was somewhat vague on this, she has had some sores in her mouth. They do heal up, but then she will get another one. She also thinks since she has been on the Lexapro, she has somewhat of a tremor of her basically whole body at least upper body including the torso and arms and had all of the daughters who I not noticed to speak of and it is certainly difficult to tell her today that she has much tremor. They do think the Lexapro has helped to some extent.",
"This 44y/o RHF awoke on 7/29/93 with left hemibody numbness without tingling, weakness, ataxia, visual or mental status change. She had no progression of her symptoms until 7/7/93 when she notices her right hand was stiff and clumsy. She coincidentally began listing to the right when walking. She denied any recent colds/flu-like illness or history of multiple sclerosis. She denied symptoms of Lhermitte's or Uhthoff's phenomena.",
"This is the initial clinic visit for a 29-year-old man who is seen for new onset of right shoulder pain. He states that this began approximately one week ago when he was lifting stacks of cardboard. The motion that he describes is essentially picking up a stack of cardboard at his waist level, twisting to the right and delivering it at approximately waist level. Sometimes he has to throw the stacks a little bit as well. He states he felt a popping sensation on 06/30/04. Since that time, he has had persistent shoulder pain with lifting activities. He localizes the pain to the posterior and to a lesser extent the lateral aspect of the shoulder. He has no upper extremity ."]

### Construct Template and Prompt through LangChain

In [3]:
#initialize template for prompt
#requests ICD-10 diagnoses with corresponding code and probability of likelihood of diagnosis
#requests medications currently/previously taken with corresponding RxCUI code
template = """
Given the following patient description, give any ICD-10 diagnoses present: '{description}'
Output the information as a table, including the probability of the diagnosis (high/medium/low). Schema: Diagnosis | ICD-10 Code | Probability
Also, include any medications that have been or are currently being used and whether the drug is currently being used or has been used and the RxNorm Concept Unique Identifier (RxCUI) code for the medication as a table. Schema: Drug Name | RxCUI | Current/Previous
"""

#construct prompt
prompt = PromptTemplate(
    input_variables=["description"],
    template=template
)

### Prompt Generation and Answers

In [4]:
#iterate over sample texts and prompt for each text
for text in range(len(texts)):
    print(f"-----------text {text}----------")
    print(llm(prompt.format(description=texts[text])))

-----------text 0----------

Diagnosis | ICD-10 Code | Probability
Poison ivy dermatitis | L23.9 | High

Drug Name | RxCUI | Current/Previous
Benadryl | 184624 | Current
Depo-Medrol | 80928 | Previous
-----------text 1----------

Diagnosis | ICD-10 Code | Probability
Allergic rhinitis | J30.9 | Low
Dermatitis of right great toe | L30.2 | High

Drug Name | RxCUI | Current/Previous
Allegra | 50613 | Previous
-----------text 2----------

Diagnosis | ICD-10 Code | Probability
Migraine | G43.909 | High
Acute sinusitis | J01.90 | Low

Drug Name | RxCUI | Current/Previous
Morphine | 611876 | Current
-----------text 3----------

Diagnosis | ICD-10 Code | Probability
Hypertension | I10 | High
Rotator Cuff Injury | M75.20 | Medium
Mouth Sores | K12.0 | Medium
Lexapro-Induced Tremor | R25.2 | Low

Drug Name | RxCUI | Current/Previous
Mavik | 65615 | Current
Verapamil | 65620 | Current
Tarka | 65621 | Previous
Aleve | 104889 | Previous
Tylenol | 103829 | Current
Tums | 103424 | Current
Mylanta | 1

### Evaluation

The previous examples are an example of a zero-shot problem setting. In this case, the model (in this case OpenAI's "text-davinci-003") was not given any special training or assisted in the way of prompt engineering. Instead, the model was tested blindly on the given task, which was to diagnose any conditions with ICD-10 codes with a level of confidence of the diagnosis, along with determining any medications which the patient has taken or is currently taken and being able to differentiate between them. 

Overall, the model did pretty well in all tasks, with the exception of identifying RxCUI codes. There is some variation in which ICD-10 codes the model may give for a specific problem, but looks like the codes are in the relevant ballpark of the diagnosis. For example, the first case example would return L23.9 (Allergic contact dermatitis, unspecified cause) or L23.7 (Allergic contact dermititis due to plants, except food). Also, the model was able to differentiate pretty well between medications currently being taken or previously, which is an important function for a model to determine in a variety of cases. This isn't the complete case, and the prompt fails to give the model an option to identify 'other' medications, such as potential. As stated, the main problem the model had was in determining the RxCUI codes for the identified medications. I was not able to determine if the codes given are even codes in the first place, as they may be an artifact of model hallucination. Perhaps the model wasn't trained on RxNorm or the prompts need better engineering.

With prompt engineering and few-shot learning utilizing real and annotated free text, I am sure that the model could perform better. And if trained further on doctor's notes, or more medical corpus (perhaps including RxNorm), then the model could perform even better. Different models could be compared as well.